In [ ]:
# Importing required libraries
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from textblob import Word
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
nltk.download('stopwords') 
nltk.download('wordnet', download_dir = "/kaggle/working/")
nltk.download('omw-1.4')
!unzip -o /kaggle/working/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
!pip install transformers

In [ ]:
td=pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv",names=["id","company","labels","text"])
vd=pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv",names=["id","company","labels","text"])

In [ ]:
df = pd.concat([td,vd])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
del df["id"]
del df["company"]

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df= df.drop_duplicates()

In [ ]:
df = df[df['labels'] != "Irrelevant"]

In [ ]:
df['labels'].value_counts()

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [ ]:
#Removing stop-words and converting words to lemma

stop_words = set(stopwords.words('english')) 

lemmatizer = WordNetLemmatizer() 

def stop_wrds_lemma_convert(sentence):
    tokens = [w for w in sentence.split() if not w in stop_words] #stopwords removal
    newString=''
    for i in tokens:                                                 
        newString=newString+lemmatizer.lemmatize(i)+' '    #converting words to lemma                               
    return newString.strip()

In [ ]:
def data_cleaning(x):
  df['tweet_clean']=x.apply(lambda x: " ".join(x.lower() for x in x.split()))#Converting them to lowercase
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub(r'@[A-Za-z0-9]+','',x))#Remove user mentions from the tweet
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub(r'[0-9]+','',x))
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub(r'http\S+','',x))
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub('#','',x))#Remove hashtags from the tweet
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in x.split(" ")]))#Remove contractions
  df['tweet_clean']=df['tweet_clean'].str.replace('[^\w\s]','')#Removing Puntuation
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: stop_wrds_lemma_convert(x))
  return df['tweet_clean']

In [ ]:
data_cleaning(df['text'])

In [ ]:
df.drop(columns=['text'],axis=0,inplace=True)

In [ ]:
df

In [ ]:
df['labels']=df['labels'].map({'Positive':1, 'Neutral':2, 'Negative':0})

In [ ]:
df['labels']

In [ ]:
X=list(df['tweet_clean'])

In [ ]:
y=list(df['labels'])

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-8_H-512_A-8')
model = BertForSequenceClassification.from_pretrained('google/bert_uncased_L-8_H-512_A-8',num_labels=3)

In [ ]:
model = model.to('cuda')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

In [ ]:
print(X_train_tokenized['attention_mask'][0])

In [ ]:
len(X_train),len(X_val)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer

args = TrainingArguments(
    report_to="none",
    output_dir="output",
    num_train_epochs=15,
    per_device_train_batch_size=32,    
    save_steps=3000,
    logging_steps=100,
    evaluation_strategy = 'epoch'
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
no_deprecation_warning=True

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
model.save_pretrained("/kaggle/working/Twitter_Sentiment_Analysis")